In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from random import choices
import scipy
import pickle
from numba import njit
from hyperopt import tpe, hp, fmin, STATUS_OK, Trials, space_eval
from hyperopt.pyll.base import scope
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.multioutput import ClassifierChain
from sklearn.feature_selection import SelectKBest, chi2, f_classif
import lightgbm as lgbm
import time
import gc
from sklearn.metrics import f1_score, log_loss, roc_auc_score, classification_report, make_scorer
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
%%time

# SEED = 1111

# np.random.seed(SEED)

train = pd.read_feather('../input/jane-street-market-train-data-best-formats/jane_street_train.feather')
train.head()

In [ ]:
train.shape

In [ ]:
%%time

train = train.query('date > 85').reset_index(drop = True) 
train = train[train['weight'] != 0]
# train = train.astype({c: np.float32 for c in train.select_dtypes(include='float64').columns})

train = train.fillna(method='ffill').fillna(0)
train = train.fillna(0)

features = [c for c in train.columns if "feature" in c]

resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp']

targets = []

for resp in resp_cols:
    name = 'action_'+resp
    train[name] = train[resp].map(lambda x: 1 if x>0 else 0)
    targets.append(name)
    
train.shape

In [ ]:
train[features].isnull().sum().sum()

In [ ]:
train[targets].mean(axis=0)

In [ ]:
train[targets].apply(lambda x: x.value_counts())

## Parameter Search

In [ ]:
# X, y = train[['date']+features], train[targets]

# X, y = reduce_mem_usage(X), reduce_mem_usage(y)

# X, y = X.values, y.values

# print(X.shape, y.shape)

In [ ]:
# def objective(params):
#     time1 = time.time()
    
#     spc = params[0]
#     n = params[1]
    
#     spc = {
#         'clf__max_depth': int(spc['clf__max_depth']),
#         'clf__subsample': spc['clf__subsample'],
#         'clf__reg_alpha': spc['clf__reg_alpha'],
#         'clf__reg_lambda': spc['clf__reg_lambda'],
#         'clf__learning_rate': spc['clf__learning_rate'],
#         'clf__num_leaves': spc['clf__num_leaves'],
#         'clf__colsample_bytree': spc['clf__colsample_bytree'],
#         'clf__min_child_samples': spc['clf__min_child_samples'],
#         'skb__k': int(spc['skb__k'])
#     }

#     print("\n############## New Run ################")
#     print(f"params = {spc}")
#     FOLDS = 5
#     count=1
#     tcv = PurgedGroupTimeSeriesSplit(n_splits=FOLDS, group_gap=20)
#     score_mean = 0
    
#     for tr_idx, val_idx in tcv.split(X=X, groups=X[:, 0]):
        
#         pipe = Pipeline(steps=[('skb', SelectKBest(f_classif)),
#                                ('clf', lgbm.LGBMClassifier(boosting_type='goss',
#                                                            n_jobs=4,
#                                                            objective='binary'))])

#         X_tr, X_vl = X[tr_idx, 1:], X[val_idx,1:]
#         y_tr, y_vl = y[tr_idx, n], y[val_idx, n]
        
#         pipe.set_params(**spc)
#         pipe.fit(X_tr, y_tr)

#         score = make_scorer(roc_auc_score, needs_proba=True)(pipe, X_vl, y_vl)

#         score_mean += score
#         print(f'{count} CV - score: {round(score, 4)}')
#         count+=1
        
#     time2 = time.time() - time1
    
#     print(f"Total Time Run: {round(time2 / 60, 2)} mins")
#     print(f'Mean ROC_AUC: {score_mean / FOLDS}')
    
#     del X_tr, X_vl, y_tr, y_vl, score
#     gc.collect()
    
#     roc_auc = -(score_mean / FOLDS)
    
#     return {'loss': roc_auc, 'status': STATUS_OK, 'model': pipe}

In [ ]:
# def getBestModelfromTrials(trials):
#     valid_trial_list = [trial for trial in trials
#                             if STATUS_OK == trial['result']['status']]
#     losses = [float(trial['result']['loss']) for trial in valid_trial_list]
#     index_having_minumum_loss = np.argmin(losses)
#     best_trial_obj = valid_trial_list[index_having_minumum_loss]
#     return best_trial_obj['result']['model']

In [ ]:
# ### t=time.time()
# param_search = []

# for n, target in enumerate(targets):
#     trials = Trials()
    
#     print("Training for target - {0}\nShape of X - {1}".format(target, X[:,1:].shape))
    
#     space = [{
#             'clf__max_depth': hp.quniform('clf__max_depth', 7, 23, 1),
#             'clf__reg_alpha':  hp.uniform('clf__reg_alpha', 0.01, 0.4),
#             'clf__reg_lambda': hp.uniform('clf__reg_lambda', 0.01, .4),
#             'clf__learning_rate': hp.uniform('clf__learning_rate', 0.01, 0.2),
#             'clf__colsample_bytree': hp.uniform('clf__colsample_bytree', 0.3, .9),
#             'clf__num_leaves': hp.choice('clf__num_leaves', list(range(20, 250, 10))),
#             'clf__min_child_samples': hp.choice('clf__min_child_samples', list(range(100, 250, 10))),
#             'clf__subsample': hp.choice('clf__subsample', [0.2, 0.4, 0.5, 0.6, 0.7, .8, .9]),
# #             'clf__random_state': hp.choice('clf__random_state', [1, 121, 1111, 12234]),
#             'skb__k': hp.quniform('skb__k', 1, len(features), 1)
#             },
#         n]

#     # Set algoritm parameters
#     best = fmin(fn=objective,
#                 space=space,
#                 algo=tpe.suggest,
#                 max_evals=30,
#                 trials=trials)

#     best_params = space_eval(space, best)
    
#     print("\nBest Parameters:\n{}".format(best_params))
    
#     best_model = getBestModelfromTrials(trials)
#     print("Best Model:\n{}".format(best_model))
    
#     preds = best_model.predict_proba(X[:,1:])
    
#     param_search.append((best_model, best_params))
#     features.append(target)
    
#     X = np.concatenate((X, preds[:,1].reshape(-1,1)), axis=1)
    
#     del trials, best, best_model, preds, space
#     gc.collect()
    
#     print("Search complete!")
#     print()
    
# print("Total time: {} mins".format((round((time.time()-t)/60), 2)))

In [ ]:
# for target, param in zip(targets, param_search):
#     print("Best params and model for target - {}".format(target))
#     print("\nBest Params - {}".format(param[1]))
#     print("\nBest Model - {}".format(param[0]))
#     print()

In [ ]:
X, y = train[features], train[targets]

X, y = reduce_mem_usage(X), reduce_mem_usage(y)

X, y = X.values, y.values

print(X.shape, y.shape)

In [ ]:
def create_model(X, y, params):
    pipe = Pipeline(steps=[('skb', SelectKBest(f_classif)),
                           ('clf', lgbm.LGBMClassifier(boosting_type='goss',
                                                        n_jobs=4,
                                                        objective='binary'))])
    
    pipe.set_params(**params)
#     print(pipe)
    
    pipe.fit(X, y)
    
    return pipe

In [ ]:
params = [
    {'clf__colsample_bytree': 0.4137747634827913, 'clf__learning_rate': 0.02201182538631583,
     'clf__max_depth': 10,
     'clf__min_child_samples': 220, 'clf__num_leaves': 90,
     'clf__reg_alpha': 0.37424061889452914, 'clf__reg_lambda': 0.39830086680073146, 
     'clf__subsample': 0.7, 'skb__k': 126},
    
    {'clf__colsample_bytree': 0.8966234519314206, 'clf__learning_rate': 0.012763372431937108, 
     'clf__max_depth': 18, 'clf__reg_alpha': 0.12053592161600853,
     'clf__min_child_samples': 100, 'clf__num_leaves': 140, 
     'clf__reg_lambda': 0.29085561719385766, 'clf__subsample': 0.7, 
     'skb__k': 117},
    
    {'clf__colsample_bytree': 0.4601256952076201, 'clf__learning_rate': 0.09877363805279632, 'clf__max_depth': 19, 
     'clf__min_child_samples': 140, 'clf__num_leaves': 20, 'clf__reg_alpha': 0.39134700340255735,
     'clf__reg_lambda': 0.17933152521594256, 'clf__subsample': 0.9, 'skb__k': 8},
    
    {'clf__colsample_bytree': 0.6234103467783765, 'clf__learning_rate': 0.010516048273273876, 'clf__max_depth': 18,
    'clf__min_child_samples': 150, 'clf__num_leaves': 40, 'clf__reg_alpha': 0.014192865855414405, 
    'clf__reg_lambda': 0.012408579091951172, 'clf__subsample': 0.5, 'skb__k': 39},
    
    {'clf__colsample_bytree': 0.8956383735014151, 'clf__learning_rate': 0.015129135845050797, 'clf__max_depth': 8,
     'clf__min_child_samples': 150, 'clf__num_leaves': 190, 'clf__reg_alpha': 0.11350501426426957, 
     'clf__reg_lambda': 0.21218839898467678, 'clf__subsample': 0.2, 'skb__k': 37}
    
]

In [ ]:
assert len(targets)==len(params)

In [ ]:
estimators = []

for n, target in enumerate(targets):
    print("Training model for target - {}".format(target))
    print(X.shape)
    pipe = create_model(X, y[:, n], params[n])
    
    preds = pipe.predict_proba(X)
    
    estimators.append(pipe)

    X = np.concatenate((X, preds[:,1].reshape(-1,1)), axis=1)
    print("Training complete!\n")

In [ ]:
X.shape

In [ ]:
del(train)
gc.collect()

In [ ]:
@njit
def fast_fillna(array, values):
    if np.isnan(array.sum()):
        array = np.where(np.isnan(array), values, array)
    return array

In [ ]:
len(features)

In [ ]:
import janestreet
env = janestreet.make_env()
env_iter = env.iter_test()

In [ ]:
th = 0.505
tmp = np.zeros(len(features))

for (test_df, pred_df) in tqdm(env_iter):
    if test_df['weight'].item() > 0:
        x_tt = test_df.loc[:, features].values
        x_tt[0, :] = fast_fillna(x_tt[0, :], tmp)
        tmp = x_tt[0, :]
        
        for est in estimators:
            preds = est.predict_proba(x_tt)[:, 1]
            x_tt = np.concatenate((x_tt, preds.reshape(-1,1)), axis=1)

        pred_df.action = np.where(preds >= th, 1, 0).astype(int)
    else:
        pred_df.action = 0
        
    env.predict(pred_df)

In [ ]:
!ls

In [ ]:
estimators[-1]